In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport
import pyarrow.feather as feather
import time
from datetime import datetime
from datetime import timedelta


In [2]:
export_df = feather.read_feather("../data/cleaned_data.feather")

In [3]:
export_df["event_timestamp"]=pd.to_datetime(export_df["event_timestamp"]*1000, unit="ns")
# utc=True

In [4]:
export_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2203480 entries, 0 to 2203479
Data columns (total 24 columns):
 #   Column                                Dtype         
---  ------                                -----         
 0   event_timestamp                       datetime64[ns]
 1   event_name                            object        
 2   user_id                               object        
 3   document_id                           object        
 4   surrogate_id                          object        
 5   created_at                            datetime64[ns]
 6   published_at                          datetime64[ns]
 7   closed_at                             datetime64[ns]
 8   cause                                 object        
 9   effect                                object        
 10  description                           object        
 11  area_of_effect_coordinates_latitude   float64       
 12  area_of_effect_coordinates_longitude  float64       
 13  agency_GewRJ

In [5]:
export_df["notif_viewed_ontime"]=list(map(lambda x,y,z: '' if x=="notification_received" else (1 if y < z else 0), export_df["event_name"], export_df["event_timestamp"],export_df["closed_at"])) #1 if the user view it on time  0 if not  and null if hasnt yet seen it 

In [6]:
export_df["reaction_time"]=list(map(lambda x,y,z,w: y-z
if (x==1 and y>z) else (y-w if (x==1 and y>w) else 0), 
export_df["notif_viewed_ontime"],export_df["event_timestamp"],export_df["published_at"],export_df["created_at"]))

In [7]:
active_df=export_df[export_df["notif_viewed_ontime"]==1].reset_index(drop=True)
unactive_df=export_df[export_df["notif_viewed_ontime"]==0].reset_index(drop=True)
active_df.drop('notif_viewed_ontime', axis=1, inplace=True)
unactive_df.drop('notif_viewed_ontime', axis=1, inplace=True)

In [8]:
export_df["notif_viewed_ontime"].value_counts()

     1294757
1     526984
0     381739
Name: notif_viewed_ontime, dtype: int64

In [9]:
export_df["reaction_time"].value_counts()

0                         1677153
0 days 00:00:18.714001          6
0 days 00:00:30.305001          6
0 days 00:00:21.825001          6
0 days 00:01:59.771001          5
                           ...   
0 days 00:04:20.433000          1
0 days 00:59:20.928000          1
0 days 01:17:13.169000          1
0 days 02:41:20.646000          1
0 days 00:03:01.242001          1
Name: reaction_time, Length: 501600, dtype: int64

In [10]:
export_df

,event_timestamp,event_name,user_id,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,...,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,reaction_time
0,2021-05-28 22:22:07.928000,notification_received,ac6d3,-Map3NyDNGaNwnfF6F0B,4Np4EQPA6xGpKOFIXMTiIg,2021-05-28 22:21:37,2021-05-28 22:21:44,2021-05-28 23:01:00,VehicularAccident,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,,0
1,2021-05-28 22:29:19.952000,notification_dismiss,ac6d3,-Map3NyDNGaNwnfF6F0B,4Np4EQPA6xGpKOFIXMTiIg,2021-05-28 22:21:37,2021-05-28 22:21:44,2021-05-28 23:01:00,VehicularAccident,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0 days 00:07:35.952000
2,2021-05-29 00:35:53.374000,notification_received,ac6d3,-MapQ4JNaTLEWbF2EG-b,kGA96xDA6xGpKOFIXMTiIg,2021-05-29 00:00:46,2021-05-29 00:21:36,2021-05-29 01:01:00,VehicularAccident,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,,0
3,2021-05-29 00:36:08.285001,notification_dismiss,ac6d3,-MapQ4JNaTLEWbF2EG-b,kGA96xDA6xGpKOFIXMTiIg,2021-05-29 00:00:46,2021-05-29 00:21:36,2021-05-29 01:01:00,VehicularAccident,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0 days 00:14:32.285001
4,2021-05-29 03:13:31.584001,notification_received,ac6d3,-Maq63vqDUxQRawQ_3Au,YKlNxSvA6xGpKOFIXMTiIg,2021-05-29 03:12:59,2021-05-29 03:15:13,2021-05-29 13:59:03,Demonstration,CirculationShutdown,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203475,2021-07-06 14:21:02.819000,notification_received,e0b4d,-MdwBMI8mNXadCw7GwTc,kCtjX2Xe6xGa3QeEZ-Zg-g,2021-07-06 14:20:53,2021-07-06 14:20:59,2021-07-06 15:21:33,Demonstration,SuspensionOfService,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,,0
2203476,2021-07-06 15:21:40.674000,notification_received,e0b4d,-MdwPCoa1U3j5wMZ8J99,YNy_023e6xGa3QeEZ-Zg-g,2021-07-06 15:21:25,2021-07-06 15:21:36,2021-07-06 20:46:48,Demonstration,SuspensionOfService,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,,0
2203477,2021-07-06 16:05:21.801000,notification_received,e0b4d,-MdwZE8hMtqpcYoaNhzM,0AF_8XPe6xGa3QeEZ-Zg-g,2021-07-06 16:05:11,2021-07-06 16:05:16,2021-07-06 17:37:59,Demonstration,SuspensionOfService,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,,0
2203478,2021-07-05 23:42:09.057001,notification_received,427c1,-Mdt2BT-2y05YXyJctt2,ADiylurd6xGf17VsriRlgg,2021-07-05 23:41:58,2021-07-05 23:42:01,2021-07-06 02:00:01,Rain,Delays,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,0


# Analyzing agencies

In [11]:
#Deeper understanding on agencies. Choosing aggfunc sum, as we want to know how many users are affected by their notifications. Observing values for both active and unactive users, it does not matter, as this is addressed in another part.
agencies_by_cause = pd.pivot_table(export_df, values=['agency_GewRJAw5tUmC4Ku4AX1-SQ', 'agency_GtvOEQAFZ0GtU6u4AXwvPg','agency_HE59N3RXM0q5vKu4AXlQZg','agency_JUR9bFXmVkWDHqu4AXaY0g', 'agency_JfA8Bw8Zp024Kqu4AXiSpQ','agency_MgUq5b9mOEunx6u4AXt_BA','agency_NuuRQ2I1Q0a50Kv-AVKlLA', 'agency_V2AIQQKgmUO3VazvAOA-Cw','agency_jLjibFoim0iwWau4AWoEdQ', 'agency_pky7jovXYkaw-awAAMrQ3g',
'agency_zCy9zG00HEqGeKu4AWZYNQ'], index=['cause'], aggfunc=np.sum, fill_value=0, margins=True, margins_name= 'Total')
agencies_by_effect = pd.pivot_table(export_df, values=['agency_GewRJAw5tUmC4Ku4AX1-SQ', 'agency_GtvOEQAFZ0GtU6u4AXwvPg','agency_HE59N3RXM0q5vKu4AXlQZg','agency_JUR9bFXmVkWDHqu4AXaY0g', 'agency_JfA8Bw8Zp024Kqu4AXiSpQ','agency_MgUq5b9mOEunx6u4AXt_BA','agency_NuuRQ2I1Q0a50Kv-AVKlLA', 'agency_V2AIQQKgmUO3VazvAOA-Cw','agency_jLjibFoim0iwWau4AWoEdQ', 'agency_pky7jovXYkaw-awAAMrQ3g',
'agency_zCy9zG00HEqGeKu4AWZYNQ'], index=['effect'], aggfunc=np.sum, fill_value=0, margins=True, margins_name= 'Total')

In [12]:
#Agencies MgUq5b9mOEunx6u4AXt_BA and V2AIQQKgmUO3VazvAOA-Cw created no incidents, dropping them
agencies_by_effect.drop(["agency_MgUq5b9mOEunx6u4AXt_BA","agency_V2AIQQKgmUO3VazvAOA-Cw"],axis=1, inplace=True)
agencies_by_cause.drop(["agency_MgUq5b9mOEunx6u4AXt_BA","agency_V2AIQQKgmUO3VazvAOA-Cw"],axis=1, inplace=True)

# Unactive users

In [13]:
unactive_df

,event_timestamp,event_name,user_id,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,...,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,reaction_time
0,2021-05-29 04:16:36.618000,notification_dismiss,8c7b7,-Maps-egC8fFLZM4HR5g,wB4ikyLA6xGpKOFIXMTiIg,2021-05-29 02:07:09,2021-05-29 02:07:12,2021-05-29 03:01:00,VehicularAccident,Delays,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2021-05-29 04:16:36.622001,notification_dismiss,8c7b7,-Maq7-D7bhcEaFBU1LDW,gDALVizA6xGpKOFIXMTiIg,2021-05-29 03:17:01,2021-05-29 03:17:12,2021-05-29 04:01:00,Incident,Delays,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2021-05-29 17:14:09.994000,notification_dismiss,8c7b7,-MasFHCt_lTuNt_x16dH,oA37g3_A6xGNq1u4NrDE6w,2021-05-29 13:12:27,2021-05-29 13:12:42,2021-05-29 14:01:00,Incident,Delays,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2021-05-29 03:17:06.023000,notification_dismiss,3f1e8,-MaphUnb8wb5IgmR7X9E,gGZoKBzA6xGpKOFIXMTiIg,2021-05-29 01:21:13,2021-05-29 01:21:34,2021-05-29 02:17:00,BrokenVehicle,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,2021-05-29 01:27:23.251000,notification_dismiss,81354,-MaogZKKMIQ_Virmn0eB,UIg1h_S_6xGvWA3t6Gmi5Q,2021-05-28 20:37:32,2021-05-28 20:38:12,2021-05-28 21:03:00,VehicularAccident,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381734,2021-07-06 12:16:54.144002,notification_opened,b6c41,-MdvT00Ih682ZROXWv-n,MOp-FUne6xGEdyPHk--PKA,2021-07-06 10:58:23,2021-07-06 10:58:26,2021-07-06 12:00:00,Incident,Delays,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
381735,2021-07-06 13:04:46.270002,notification_opened,b6c41,-MdvkXIakRmEyb5NrwhW,YJhCY1Te6xGEdyPHk--PKA,2021-07-06 12:19:18,2021-07-06 12:19:25,2021-07-06 12:50:00,VehicularAccident,LaneReduction,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
381736,2021-07-05 23:05:16.648000,notification_opened,7a79c,-Mdsb6FPBr0BBEK8UFBY,oHf4ctnd6xGfY3t29ZPLVg,2021-07-05 21:39:17,2021-07-05 21:39:20,2021-07-05 22:40:01,Incident,Delays,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
381737,2021-07-06 17:08:21.329002,notification_opened,0bf97,-Mdwhnz9Hyl6NGRfhlhM,oAKtyXne6xGa3QeEZ-Zg-g,2021-07-06 16:47:02,2021-07-06 16:47:39,NaT,Demonstration,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [14]:
unactive_df.columns

Index(['event_timestamp', 'event_name', 'user_id', 'document_id',
       'surrogate_id', 'created_at', 'published_at', 'closed_at', 'cause',
       'effect', 'description', 'area_of_effect_coordinates_latitude',
       'area_of_effect_coordinates_longitude', 'agency_GewRJAw5tUmC4Ku4AX1-SQ',
       'agency_GtvOEQAFZ0GtU6u4AXwvPg', 'agency_HE59N3RXM0q5vKu4AXlQZg',
       'agency_JUR9bFXmVkWDHqu4AXaY0g', 'agency_JfA8Bw8Zp024Kqu4AXiSpQ',
       'agency_MgUq5b9mOEunx6u4AXt_BA', 'agency_NuuRQ2I1Q0a50Kv-AVKlLA',
       'agency_V2AIQQKgmUO3VazvAOA-Cw', 'agency_jLjibFoim0iwWau4AWoEdQ',
       'agency_pky7jovXYkaw-awAAMrQ3g', 'agency_zCy9zG00HEqGeKu4AWZYNQ',
       'reaction_time'],
      dtype='object')

In [15]:
unactive_analysis_df=unactive_df.drop(['agency_GewRJAw5tUmC4Ku4AX1-SQ', 'agency_GtvOEQAFZ0GtU6u4AXwvPg',
       'agency_HE59N3RXM0q5vKu4AXlQZg', 'agency_JUR9bFXmVkWDHqu4AXaY0g',
       'agency_JfA8Bw8Zp024Kqu4AXiSpQ', 'agency_MgUq5b9mOEunx6u4AXt_BA',
       'agency_NuuRQ2I1Q0a50Kv-AVKlLA', 'agency_V2AIQQKgmUO3VazvAOA-Cw',
       'agency_jLjibFoim0iwWau4AWoEdQ', 'agency_pky7jovXYkaw-awAAMrQ3g',
       'agency_zCy9zG00HEqGeKu4AWZYNQ', 'surrogate_id','document_id'], axis=1)

In [16]:
unactive_analysis_df

,event_timestamp,event_name,user_id,created_at,published_at,closed_at,cause,effect,description,area_of_effect_coordinates_latitude,area_of_effect_coordinates_longitude,reaction_time
0,2021-05-29 04:16:36.618000,notification_dismiss,8c7b7,2021-05-29 02:07:09,2021-05-29 02:07:12,2021-05-29 03:01:00,VehicularAccident,Delays,⌛ ¡No de nuevo! Hay retrasos por accidente ve...,-99.170952,19.393843,0
1,2021-05-29 04:16:36.622001,notification_dismiss,8c7b7,2021-05-29 03:17:01,2021-05-29 03:17:12,2021-05-29 04:01:00,Incident,Delays,⏳ ¡Pa variar! Usuarios reportan retrasos de ha...,NaN,NaN,0
2,2021-05-29 17:14:09.994000,notification_dismiss,8c7b7,2021-05-29 13:12:27,2021-05-29 13:12:42,2021-05-29 14:01:00,Incident,Delays,⏳ ¡No te esponjes! Usuarios reportan retrasos ...,NaN,NaN,0
3,2021-05-29 03:17:06.023000,notification_dismiss,3f1e8,2021-05-29 01:21:13,2021-05-29 01:21:34,2021-05-29 02:17:00,BrokenVehicle,Delays,🚗 ¡Sopas! Retrasos por vehículo descompuesto e...,-99.066870,19.404783,0
4,2021-05-29 01:27:23.251000,notification_dismiss,81354,2021-05-28 20:37:32,2021-05-28 20:38:12,2021-05-28 21:03:00,VehicularAccident,Delays,⌛ ¡Ah jijo! Hay retrasos por accidente vehicul...,-99.082618,19.421229,0
...,...,...,...,...,...,...,...,...,...,...,...,...
381734,2021-07-06 12:16:54.144002,notification_opened,b6c41,2021-07-06 10:58:23,2021-07-06 10:58:26,2021-07-06 12:00:00,Incident,Delays,⏳ ¡Changos! Usuarios informan retrasos de más ...,NaN,NaN,0
381735,2021-07-06 13:04:46.270002,notification_opened,b6c41,2021-07-06 12:19:18,2021-07-06 12:19:25,2021-07-06 12:50:00,VehicularAccident,LaneReduction,⏳ ¡Chanfle! Reducción de carriles por percance...,-99.059346,19.304670,0
381736,2021-07-05 23:05:16.648000,notification_opened,7a79c,2021-07-05 21:39:17,2021-07-05 21:39:20,2021-07-05 22:40:01,Incident,Delays,"⏳ No te agüites, pero usuarios reportan retras...",NaN,NaN,0
381737,2021-07-06 17:08:21.329002,notification_opened,0bf97,2021-07-06 16:47:02,2021-07-06 16:47:39,NaT,Demonstration,Delays,✋ ¡Chanclas! Hay cierre de circulación por man...,-99.041880,19.343449,0


In [17]:
unactive_analysis_df['created_at'].sort_values(ascending=True)


96133    2021-01-01 00:24:50
90882    2021-01-01 00:24:50
90879    2021-01-01 00:24:50
90883    2021-01-01 02:11:51
90877    2021-01-01 02:11:51
                 ...        
355943   2021-07-07 03:47:43
18611    2021-07-07 03:47:43
356256   2021-07-07 03:47:43
355934   2021-07-07 03:47:43
356347   2021-07-07 03:47:43
Name: created_at, Length: 381739, dtype: datetime64[ns]

In [18]:
unactive_analysis_df.query('created_at=="NaT"')


,event_timestamp,event_name,user_id,created_at,published_at,closed_at,cause,effect,description,area_of_effect_coordinates_latitude,area_of_effect_coordinates_longitude,reaction_time


In [19]:
export_df[export_df["created_at"].isnull()]

,event_timestamp,event_name,user_id,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,...,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,reaction_time
